# **Practical assignment for Topic 9**

In the assignment, you have to use the dataset `auto-lpkm.tsv` (with quadratic features) given in the lecture and you should reuse the given code.

Your assignment consists of **three tasks**:
<br>
<br>

**Task 1**<br>
In function `GA()`, implement tournament selection. Set tournament size to 4.<br>
Do not remove the roulette wheel selection. Instead write the code so that it is easy to choose which selection method to use - from the arguments of the `GA()` function.
<br>
<br>

**Task 2**<br>
In function `GA()`, implement <a href="https://en.wikipedia.org/wiki/Crossover_(genetic_algorithm)#Uniform_crossover">uniform cross-over</a>. As discussed in the lecture, for each gene there should be 50% probability to go to the first child and other 50% probability to go to the second child. And then the other parent's gene does the opposite.<br>
Do not remove the single-point cross-over. Instead write the code so that it is easy to choose which cross-over method to use - from the arguments of the `GA()` function.
<br>
<br>

**Task 3**<br>
Evaluate the following four configurations of GA without any other configuration changes:
* Configuration 1: Roulette wheel selection and single-point cross-over.
* Configuration 2: Roulette wheel selection and uniform cross-over.
* Configuration 3: Tournament selection and single-point cross-over.
* Configuration 4: Tournament selection and uniform cross-over.

For the four evaluations, run the function on the given dataset and take note of the finally computed $R^2$ value. To do it properly, run each configuration 10 times and compute the averate $R^2$. This will help to reduce the uncertainty of the evaluations. Now compare the numbers and write conclusions.

In addition, also compare four "fitness vs. generation" diagrams - one diagram from each configuration's single run. Or if you are able to manage the plot's visual complexity, you may try to show all runs from all configurations in one plot. Finally, write conclusions about this comparison as well.

---
**After the tasks are done, submit this file. Do not clear it's output - all print-outs and diagrams should be left in the file.**